**Inportando dados**

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = os.path.abspath(os.path.join(os.pardir))
DATA_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')
sys.path.append(os.path.join(PROJECT_ROOT, 'src'))

from utils.dates import Dates

**Tabela Projetos**

In [2]:
projetos_csv = os.path.join(DATA_FOLDER, 'projetos.csv')

cols = ['IdPRONAC', 'DtProtocolo', 'Situacao']

dt_projetos = pd.read_csv(projetos_csv, usecols = cols, low_memory = False)

dt_projetos.head()

,IdPRONAC,Situacao,DtProtocolo
0,1,E63,2008-09-12 00:00:00
1,2,G43,2008-09-12 00:00:00
2,3,K00,2008-09-12 00:00:00
3,4,E25,2008-09-03 00:00:00
4,5,E36,2008-09-12 00:00:00


**Separar projetos criados e projetos finalizados por ano**

In [3]:
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
end_situations = ['A09', 'A13', 'A14', 'A16', 'A17', 'A18', 'A20', 'A23', 'A24', 'A26', 'A40', 'A41', 'A42', 
                  'C09', 'D18', 'E04', 'E07', 'E09', 'E47', 'E49', 'E63', 'E64', 'E65', 'G16', 'G25', 'G26',
                  'K00', 'K01', 'K02', 'L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L08', 'L10', 'L11']

created_projects = []
finished_projects = []

for year in years:
    project_created_after_begin_year = dt_projetos['DtProtocolo'] > '{}-01-01'.format(year)
    project_created_before_end_year = dt_projetos['DtProtocolo'] < '{}-12-31'.format(year)
    
    created_projects_filter = project_created_after_begin_year & project_created_before_end_year
    finished_projects_filter = created_projects_filter & dt_projetos['Situacao'].isin(end_situations)
    
    created_projects.append(len(dt_projetos[created_projects_filter]))
    finished_projects.append(len(dt_projetos[finished_projects_filter]))

    
created_projects_percentage = [0] + [(created_projects[i] - created_projects[i - 1]) * 100 / created_projects[i - 1] for i in range(1, len(created_projects))]
finished_projects_percentage = [0] + [(finished_projects[i] - finished_projects[i - 1]) * 100 / finished_projects[i - 1] for i in range(1, len(finished_projects))]

**Tabela de crescimento de projetos criados e finalizados**

In [11]:
def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color


data = {'Ano': years, 'Projetos Criados': created_projects, 'Projetos Criados %': created_projects_percentage,
        'Projetos Fechados': finished_projects, 'Projetos Fechados %': finished_projects_percentage}

df = pd.DataFrame(data = data)

subset = pd.IndexSlice[:, ['Projetos Criados %', 'Projetos Fechados %']]

df = df.style.applymap(color_negative_red, subset = subset)

display(df)

,Ano,Projetos Criados,Projetos Criados %,Projetos Fechados,Projetos Fechados %
0,2009,9181,0,5159,0
1,2010,13532,47.3914,6863,33.0297
2,2011,14068,3.96098,5811,-15.3286
3,2012,10016,-28.803,4759,-18.1036
4,2013,11534,15.1558,5635,18.4072
5,2014,14419,25.013,5294,-6.05146
6,2015,11274,-21.8115,2795,-47.2044
7,2016,5097,-54.7898,1269,-54.5975
8,2017,9422,84.8538,3058,140.977
9,2018,708,-92.4857,6,-99.8038
